Questions.. how do I predict macro trends for Crypto currencies 

### Get the dates and amount of fear greed  

In [360]:
import pandas as pd
savedf = pd.read_csv(r"./Fear Greed Index.csv")

# Dropping rows
savedf = savedf.drop(columns='Unnamed: 1')
savedf = savedf.drop(columns='Unnamed: 2')
savedf = savedf.drop(columns='Unnamed: 3')
savedf = savedf.drop(columns='Unnamed: 4')
savedf = savedf.drop(columns='Unnamed: 5')
savedf = savedf.drop(savedf.tail(5).index)

# Renaming columns 
savedf = savedf.rename(columns={'Unnamed: 0': 'c1'})

### Split the date and the amount into two seporate columns 

In [361]:
# totally not relavent to the rest of the analysis
# df['c2'] = df.iloc[::2,0]
# df['c3'] = df.iloc[1::2,0]

# df

### Drop NaNs

In [362]:
df = savedf

df = pd.DataFrame(df.c1.values.reshape(-1,2),columns=['c2','c3'])
df = df.rename(columns={'c2': 'date'})
df = df.rename(columns={'c3': 'sentiment'})

savedf = df

### strip the words from C2

In [363]:
df = savedf

df['sentiment'] = df['sentiment'].str[0:2]
df = df.replace(',','', regex=True)
df['sentiment'] = pd.to_numeric(df['sentiment'])

savedf = df

#### Add btc price 

In [364]:
btcPrice = pd.read_csv(r"./btcPrice.csv")
btcPrice = btcPrice.drop(columns=['Open', 'High', 'Low', 'Volume', 'Adj Close'])
btcPriceSave = btcPrice

#### Merge BTC prices with dataframe    

In [365]:
btcPrice = btcPriceSave

df = savedf

df.date = pd.to_datetime(df.date)
btcPrice.Date = pd.to_datetime(btcPrice.Date)
btcPrice = btcPrice.rename(columns={'Date': 'date'})
btcPrice = btcPrice.rename (columns={'Close': 'price'})
df = pd.merge(btcPrice, df, how='outer', on='date')
df = df.dropna()

savedf = df

#### Intersect both graphs over each other 

In [516]:
df = savedf

import numpy as np
import matplotlib.pyplot as plt

# def halfSpreadsheet(df):
#     df = df.drop(df.tail(1).index)
#     df = df.drop(df.head(1).index)
#     return df   

# slicing function
# df = df.iloc[np.r_[0:1000]]

def trailingAverage(df, n):
    df['avg'] = df['sentiment'].rolling(n).mean()
    return df

def replaceNaN(x):
    for i in range(len(x)):
        if np.isnan(x[i]):
            x[i] = 0
    return x

def turnNumbersToPerecentes(x):
    for i in range(len(x)):
        x[i] = ("{:.2%}".format( x[i]/ 100)) 

    return x

date = df['date'].tolist()
sentiment = df['sentiment'].tolist()
price = df['price'].tolist()
avg = df['avg'].tolist()
avg = replaceNaN(avg)
# avg = turnNumbersToPerecentes(avg)

correlation = df['price'].corr(df['avg'])
correlation = ("{:.2%}".format( correlation))
correlation

# print(avg[-50:])
# plt.rcParams["figure.figsize"] = (20,5) 

# x = np.arange(0, 10, 0.1)
# y1 = 0.05 * x**2
# y2 = -1 *y1

# fig, ax1 = plt.subplots()

# ax2 = ax1.twinx()
# ax1.plot(date, price, 'g-')
# ax2.plot(date, avg, 'y-', linewidth=.5)

# ax1.set_xlabel('date')
# ax1.set_ylabel('price', color='g')
# ax2.set_ylabel('avg', color='y')


'36.84%'